In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import pickle

# Load and preprocess data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train, X_test = X_train.reshape(-1, 28, 28, 1), X_test.reshape(-1, 28, 28, 1)
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# Define CNN Model
cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)
cnn.save("models/cnn_model.h5")

# Define LeNet Model
lenet = Sequential([
    Conv2D(6, (5,5), activation='relu', padding='valid', input_shape=(28,28,1)),
    AveragePooling2D(pool_size=(2,2)),
    Conv2D(16, (5,5), activation='relu', padding='valid'),
    AveragePooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(10, activation='softmax')
])

lenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lenet.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)
lenet.save("models/lenet_model.h5")



/home/solitary-developer/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-30 22:41:11.337108: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/5


2025-01-30 22:41:11.823772: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 169344000 exceeds 10% of free system memory.


1688/1688 ━━━━━━━━━━━━━━━━━━━━ 31s 18ms/step - accuracy: 0.9031 - loss: 0.3191 - val_accuracy: 0.9793 - val_loss: 0.0695
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 29s 17ms/step - accuracy: 0.9823 - loss: 0.0587 - val_accuracy: 0.9850 - val_loss: 0.0604
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 29s 17ms/step - accuracy: 0.9897 - loss: 0.0348 - val_accuracy: 0.9873 - val_loss: 0.0445
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9931 - loss: 0.0231 - val_accuracy: 0.9882 - val_loss: 0.0486
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 29s 17ms/step - accuracy: 0.9950 - loss: 0.0157 - val_accuracy: 0.9890 - val_loss: 0.0486


Epoch 1/5


2025-01-30 22:43:50.661498: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 169344000 exceeds 10% of free system memory.


1688/1688 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.8460 - loss: 0.4935 - val_accuracy: 0.9765 - val_loss: 0.0815
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9745 - loss: 0.0834 - val_accuracy: 0.9795 - val_loss: 0.0692
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.9802 - loss: 0.0633 - val_accuracy: 0.9853 - val_loss: 0.0525
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9840 - loss: 0.0496 - val_accuracy: 0.9833 - val_loss: 0.0577
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9874 - loss: 0.0404 - val_accuracy: 0.9852 - val_loss: 0.0518


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import pickle

# Load dataset
train_data = pd.read_csv("./dataset/mnist_train.csv")
test_data = pd.read_csv("./dataset/mnist_test.csv")
dataset = pd.concat([train_data, test_data], ignore_index=True)

X, Y = dataset.iloc[:, 1:], dataset.iloc[:, 0]

# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

# Apply PCA
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train KNN Model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_pca, y_train)

# Save Model
with open("models/pca_knn.pkl", "wb") as f:
    pickle.dump(knn, f)

with open("models/pca.pkl", "wb") as f:
    pickle.dump(pca, f)



In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

# Load dataset
train_data = pd.read_csv("./dataset/mnist_train.csv")
test_data = pd.read_csv("./dataset/mnist_test.csv")
dataset = pd.concat([train_data, test_data], ignore_index=True)

X, Y = dataset.iloc[:, 1:], dataset.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train Random Forest Model
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
print("Random Forest Accuracy:", accuracy_score(y_test, rfc.predict(X_test)))

# Save Model
with open("models/random_forest.pkl", "wb") as f:
    pickle.dump(rfc, f)



Random Forest Accuracy: 0.9669285714285715


In [9]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pickle

# Load dataset
train_data = pd.read_csv("./dataset/mnist_train.csv")
test_data = pd.read_csv("./dataset/mnist_test.csv")
dataset = pd.concat([train_data, test_data], ignore_index=True)

X, Y = dataset.iloc[:, 1:], dataset.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train SVM Model
svm = SVC(kernel="poly", C=1.0)
svm.fit(X_train, y_train)

# Save Model
with open("models/svm.pkl", "wb") as f:
    pickle.dump(svm, f)

